<a href="https://www.kaggle.com/code/joshuaokolo/crop-yield-prediction-app?scriptVersionId=103970139" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Project Overview

This project utilize Google Earth Engine to acquire satellite images from the user selects on the interactive map.

In [2]:
!pip install geemap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 1.8 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.1/198.1 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.8/99.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 11.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 4.9 MB/s eta 0:00:00
  Created wheel for ee-extra: filename=ee_extra-0.0.14-py3-none-any.whl si

In [6]:
import geemap
Map = geemap.Map()

In [9]:
#draw a rectangle in the above map and then run this
#ROI - Region of Interest
feature = Map.draw_last_feature
ROI = feature.geometry()
ROI.getInfo()

## Download geo tiff file

In [ ]:
import ee

In [ ]:
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000) 


# Load Sentinel-2 TOA reflectance data.
imageCollection = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2016-01-01', '2016-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)) \
    .map(maskS2clouds) \
    .filterBounds(roi) \
    .sort('CLOUDY_PIXEL_PERCENTAGE')


rawImg = ee.Image(imageCollection.toList(imageCollection.size()).get(0))

#get the matrix for B2 band only
B2_image = geemap.ee_to_numpy(rawImg.select(['B2']), region=ROI)

In [ ]:
targetSize = (50,50)

In [ ]:
B5_Matrix = cv2.resize(B5_image,targetSize,interpolation=cv2.INTER_NEAREST)

# Make App

## Define UI

In [ ]:
#Date picker 
dateBox = widgets.DatePicker(
    description='',
    disabled=False
)

#Map
mapWidget = widgets.Output()
    
#labels
step1 = widgets.Label('Step 1: Select the date')
step2 = widgets.Label('Step 2: Select Region from the map')
step3 = widgets.Label('Step 3: Load model')
step4 = widgets.Label('Step 4: Estimate yield')
estimate_yield_debug = widgets.Label('')

In [ ]:
#Buttons
getROI = widgets.Button(description='Ok')
estimate_yield_Btn = widgets.Button(description='Estimate yield')
loadModel_Btn = widgets.Button(description='Load model')

#Progress bar
progressBar = widgets.IntProgress(
    value=0,
    min=0,
    max=19,
    step=1,
    description='',
    bar_style='info', 
    orientation='horizontal'
)

#Text Area
estimate_yield_Out = widgets.Textarea(
    value='',
    placeholder='',
    description='',
    disabled=True,
    layout={'height': '300px'}
)

#Display prediction image (Matplotlib plot)
estimate_yield_plot = widgets.Output()

# Update widgets

## Update label

In [ ]:
for band in bands:
    estimate_yield_debug.value = f"Processing: {band}"

## Read info from date picker

In [ ]:
year  = dateBox.value.year
month = dateBox.value.month
day   = dateBox.value.day

## Display app

In [ ]:
Map = geemap.Map(center=[14.607989,-14.531731], zoom=7)
Map.add_basemap('Google Satellite Hybrid')

In [ ]:
with mapWidget:
    display(Map)

## To update the progress bar

In [ ]:
def getStatistics(change):
      """
      code
      """    
getROI.on_click(getStatistics)

In [ ]:
for band in bands:
      estimate_yield_debug.value = f"Processing: {band}"
      #Update progress bar
      progressBar.value = progressBar.value +1

## Display matplot image

In [ ]:
def plotResult(prediction):
    """
    Code to visualize the prediction in the form of Matplotlib image.
    """

    
    #display output
    plt.show()

In [ ]:
#Visualize the prediction
with estimate_yield_plot:
      plotResult(prediction)

## Create app layout

In [ ]:
from ipywidgets import AppLayout

In [ ]:
#Arrange the layout
verticalBox = mapWidget
vBox1 = widgets.VBox([step1, dateBox,vBox])
vBox2 = widgets.VBox([step2, getROI,
                      step3,loadModel_Btn,                            step4,estimate_yield_Btn,progressBar,estimate_yield_debug,yieldOutput,
                      estimate_yield_Out,estimate_yield_plot])

AppLayout(header=None,
          left_sidebar=vBox1,
          right_sidebar=vBox2,
          footer=None,
          height="70%", width="150%")

# To run the application

Open the notebook on your local jupyter app. Click on the Voilà button.

## Credits

Inspired by Praveen Kanna, [here](https://omdena.com/blog/yield-prediction/).